In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc
import hvplot
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
from holoviews import opts

#### Read tide data

In [ ]:
tide_file = '/home/jovyan/data/tide/pred_F.txt'
df_tides = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides['datetime']=df_tides['year'] + '-' + df_tides['month'] + '-' + df_tides['day'] + \
            'T' + df_tides['hour'] + ':' + df_tides['minute'] + ':' + df_tides['second']
df_tides.index=pd.to_datetime(df_tides['datetime'].values)
del df_tides['year']
del df_tides['month']
del df_tides['day']
del df_tides['hour']
del df_tides['minute']
del df_tides['second']
del df_tides['datetime']
df_tides['height'] = df_tides['height'].astype(float)
df_tides= df_tides.sort_index()

In [ ]:
#list comprehention
epoch= [i.timestamp() for i in df_tides.index.to_pydatetime()]

In [ ]:
df_tides['epoch'] = epoch

### Pull in data MJ03F

In [ ]:
with open('/home/jovyan/data/botpt/2019bottom_pressure15s_F.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptF = pd.DataFrame(botpt_data)
df_botptF['bottom_pressure'] = df_botptF['bottom_pressure'].astype(float)
df_botptF['depth']=df_botptF['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention"
epoch= [i.timestamp() for i in df_botptF.index.to_pydatetime()]
df_botptF['epoch'] = epoch
df_botptF= df_botptF.sort_index()
df_botptF.index.name= 'Date'
df_botptF.tail()

In [ ]:
df_botptF.hvplot.scatter(x='Date', y='depth', height=200, datashade=True,
                       flip_yaxis=True)

In [ ]:
df_subsetTides = df_tides.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']
df_subsetBOTPTF = df_botptF.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']

In [ ]:
df_subsetBOTPTF['tides'] = df_subsetTides.height
df_subsetBOTPTF.tail()

In [ ]:
df_subsetBOTPTF['dtide']=  df_subsetBOTPTF['depth'] - df_subsetBOTPT['tides']
df_subsetBOTPTF.tail()

In [ ]:
df_subsetBOTPTF.hvplot(x='Date', y='dtide', datashade=True, height=200,
                       flip_yaxis=True)